![Département de Mathématiques](https://ktzanev.github.io/logolabopp/math-ulille/math-ulille_100.gif)

# TP 1 - Les équations différentielles de dimension 1

Dans ce premier TP nous allons étudier numériquement des équations du premier ordre de dimension 1.<br>
Il s'agit d'exemples vus dans la **fiche 2** de TD.

**Attention :** Pour que le code entre les exercices soit au maximum similaire et permettre de faire des copier/coller, nous utilisons plusieurs fois les mêmes noms pour des fonctions différentes. **Ceci est une très mauvaise pratique qui mène à des résultats imprévisibles si on n'exécute pas les cellules dans l'ordre.**

On commence par charger les bibliothèques « standards » dont on aura besoin.

In [ ]:
# numpy pour les calculs (vectoriels)
import numpy as np
# matplotlib (référencé comme `plt` ici) pour les graphiques
from matplotlib import pyplot as plt
# la bibliothèque pour tracer les trajectoires
from scipy.integrate import odeint
# la bibliothèque qui permet d'utiliser (le décorateur) @interact
from ipywidgets import interact

## Une fonction utile

La fonction `champ_normalise` permet de tracer le champ de vecteur $(1,f(y,t))$ **normalisé** associé au problème de Cauchy $y'=f(y,t)$.
La fonction `f` passé en paramètre doit accepter deux **vecteurs** `y` et `t` comme paramètres. Autrement dit elle doit être définie avec `f = lambda y,t : ...` ou avec `def f(y,t):` où `y` et `t` sont des tableaux `numpy` (ou des nombres).

**Attention :** L'ordre des paramètres de `f` est inversé par rapport aux cours et les tds :
- en cours et td : $y'(t) = f(t,y(t))$ _(temps, puis variable spatiale)_
- en python (`odeint`) : `f(y,t)` _(variable spatiale, puis temps)_



In [ ]:
# affiche le champ normalisé sur la figure courante
# - f représente la fonction de y'=f(y,t)
# - tmin, tmax, ymin, ymax est la fenêtre d'affichage
# - N est le nombre de discrétisations en chaque direction
def champ_normalise(f, tmin, tmax, ymin, ymax, N=15, **kw):
    F = lambda Y: [np.ones_like(Y[0]), f(Y[1], Y[0])]  # la fonction du problème autonome associé
    T = np.linspace(tmin, tmax, N)  # abscisses des points de la grille
    Y = np.linspace(ymin, ymax, N)  # ordonnées des points de la grille
    U, V = F(np.meshgrid(T, Y))  # les composantes du champ de vecteurs
    M = np.hypot(U, V)  # calcule la norme du vecteur (U,V)
    M[M == 0] = 1  # évite la division par 0
    U /= M  # normalise la composante U
    V /= M  # normalise la composante V
    return plt.quiver(T, Y, U, V, angles='xy', **kw)  # trace le champ de vecteurs sur la grille de NxN points

## Exercice 2 (exemple)

Commençons par illustrer l'utilisation de python avec l'exercice 2 de la feuille de TD n°2. On vous rappelle que dans cet exercice on a étudié l'équation
$$
    y'=|y|+|t|
$$
qui est de la forme de Cauchy $y'=f(y,t)$ avec $f(y,t) =|y|+|t|$.

Nous allons définir et tester la fonction $f$ de la façon suivante :

In [ ]:
# la fonction f
f = lambda y, t: np.abs(y) + np.abs(t)

# test
assert f(0, 0) == 0, "|0]+|0] = 0"
assert f(-3, 1) == 4, "|-3]+|1] = 4"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

Pour dessiner le champ de vecteurs associé à ce problème de Cauchy non autonome de dimension 1, on place $t$ sur abscisse et $y$ sur l'ordonnée et on utilise la fonction `champ_normalise` définie précédemment.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# taille de la figure
plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
# on détermine la fenêtre d'affichage
plt.axis([tmin, tmax, ymin, ymax])
# on trace les axes en `yellow`
plt.plot([tmin, tmax], [0, 0], "y")
plt.plot([0, 0], [ymin, ymax], "y")

# on dessine le champ normalisé
champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

# on rajoute le titre
plt.title(f"Le champ de vecteurs.")
# et on affiche tout ça
plt.show()

### La solution avec CI $y(0)=0$

Nous avons vu en TD que la solution (appelée ici $s$) qui vérifie la condition initiale $y(0)=0$ est
$$
s : t\mapsto 
\begin{cases}
    -t-1+e^t    & \text{ si } t\geq 0\\
    -t+1-e^{-t} & \text{ si } t\leq 0\\
\end{cases}.
$$

On peut encoder cette fonction ainsi :

In [ ]:
s = lambda t: np.piecewise(t, [t < 0, t >= 0], [lambda t: -t + 1 - np.exp(-t), lambda t: -t - 1 + np.exp(t)])

Pour dessiner le graphe de cette fonction on utilise la commande `plt.plot`.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# taille de la figure
plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
# on détermine la fenêtre d'affichage
plt.axis([tmin, tmax, ymin, ymax])
# on trace les axes en `yellow`
plt.plot([tmin, tmax], [0, 0], "y")
plt.plot([0, 0], [ymin, ymax], "y")

# la solution que nous avons trouvée en TD pour (0,0)
t = np.linspace(tmin, tmax, 100)
plt.plot(t, s(t), "r")

plt.title(f"La solution avec y(0)=0.")
# et on affiche tout ça
plt.show()

### La solution avec CI $y(0) = y_0$

Nous n'avons pas décrit en TD la solution avec la condition initiale plus générale $y(0) = y_0$, même si c'est possible. Ici nous allons profiter de la fonction `odeint` pour tracer cette solution pour différentes valeurs de $y_0$. Commençons par le cas $y_{0}=1$. Dans ce cas particulier la solution est affine pour $t\in[-1,0]$, et nous pouvons vérifier ceci visuellement en dessinant la droite de pente $1$ qui passe par $(-1,0)$.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# choix de la condition initiale
t_0 = 0
y_0 = 1

# taille de la figure
plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
# on détermine la fenêtre d'affichage
plt.axis([tmin, tmax, ymin, ymax])
# on trace les axes en `yellow`
plt.plot([tmin, tmax], [0, 0], "y")
plt.plot([0, 0], [ymin, ymax], "y")

# la droite de pente 1 qui passe par (-1,0) pour comparer avec la solution
t = np.linspace(tmin, tmax, 2)
plt.plot(t, t + 1, "lightgray")

# La solution approchée, trouvée avec odeint
# la partie de la courbe pour t dans [t_0,tmax] (le futur)
futur = np.linspace(t_0, tmax, 50)
v = odeint(f, y_0, futur)
plt.plot(futur, v, "b")
# la partie de la courbe pour t dans [tmin,t_0] (le passé)
passe = np.linspace(t_0, tmin, 50)
v = odeint(f, y_0, passe)
plt.plot(passe, v, "g--")

# le point de départ (la condition initiale)
plt.plot(t_0, y_0, 'or')
# le titre
plt.title(f"Solution avec y({t_0})={y_0}.")
# et on affiche tout ça
plt.show()

### Le graphique en version « statique »

Nous somme prêt maintenant pour rassembler les différents éléments sur une même image. Pour cela nous allons créer une fonction `solution(t_0, y_0)` qui prend les paramètre $t_0$ et $y_0$ et qui dessine sur le même graphique :
- le champ de vecteurs ;
- la solution exacte $s$ qui vérifie $y(0)=0$ ;
- la solution approchée obtenue avec `odeint` qui vérifie $y(t_0) = y_0$.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -2, 2, -2, 2

# le dessin pour la condition initiale (t_0,y_0)
def solution(t_0=0, y_0=0):
    # taille de la figure
    plt.figure(figsize=(2 * (tmax - tmin), 2 * (ymax - ymin)))
    # on détermine la fenêtre d'affichage
    plt.axis([tmin, tmax, ymin, ymax])
    # on trace les axes en `yellow`
    plt.plot([tmin, tmax], [0, 0], "y")
    plt.plot([0, 0], [ymin, ymax], "y")

    # on dessine le champ normalisé
    champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

    # la solution que nous avons trouvée en TD pour (0,0)
    t = np.linspace(tmin, tmax, 100)
    plt.plot(t, s(t), "r")

    # La solution approchée, trouvée avec odeint
    # la partie de la courbe pour t dans [t_0,tmax] (le futur)
    futur = np.linspace(t_0, tmax, 50)
    v = odeint(f, y_0, futur)
    plt.plot(futur, v, "b")
    # la partie de la courbe pour t dans [tmin,t_0] (le passé)
    passe = np.linspace(t_0, tmin, 50)
    v = odeint(f, y_0, passe)
    plt.plot(passe, v, "g--")

    # le point de départ (la condition initiale)
    plt.plot(t_0, y_0, 'or')
    # le titre
    plt.title(f"Solution avec y({t_0})={y_0}.")
    # et on affiche tout ça
    plt.show()

# vérifions que ça fonctionne
solution(0, 1)

### Le graphique en version « dynamique »

Maintenant nous pouvons facilement rendre dynamique le graphique précédent en faisant varier $t_0$ et $y_0$ grâce au décorateur `@interact`.

In [ ]:
# choix de la condition initiale grâce à des curseurs
@interact(t_0=(tmin, tmax, .5), y_0=(ymin, ymax, .5))
# le dessin pour la condition initiale (t_0,y_0)
def dyn_solution(t_0=0, y_0=0):
    solution(t_0, y_0)

## Exercice 3 (exemple)

Notre équation sera ici celle de l'exercice 3 de la feuille 2 
$$
    y'=|y|^{\frac{2}{3}} 
$$
la fonction $f$ à définir est $f(y,t) = |y|^{\frac{2}{3}}$.

In [ ]:
# la fonction f
f = lambda y, t: np.cbrt(y**2)  # ici `np.cbrt` est la racine cubique

# test
assert f(0, 0) == 0, "En y=0 la fonction doit valoir 0"
assert f(1, 0) == 1, "En y=1 la fonction doit valoir 1"

### Les solutions

Comme nous avons vu en TD cette équation ne vérifie pas la condition de Cauchy-Lipschitz. Nous avons vu qu'il y a une infinité de solutions qui vérifient $y(0)=0$, obtenues en recollant la fonction constante $y=0$ et une fonction
$$
    s_{t_0}(t) = \left(\frac{t-t_0}{3}\right)^3
$$
vérifie aussi la condition initiale $s(t_0)=0$ pour $t_0 \geq 0$.

Pour définir cette fonction $s_{t_0}$ dépendant de paramètre $t_0$, en python nous allons définir une fonction `ss(t_0)` qui va retourner la fonction recherchée. Ainsi nous allons pouvoir faire `s = ss(1)`, par exemple, puis `s(1)` devrait valoir `0`.

In [ ]:
# ss(t_0) retourne la fonction non triviale qui vérifie s(t_0) = 0
ss = lambda t_0: lambda t: ((t - t_0) / 3.)**3

assert ss(1)(1) == 0, "La fonction ss(1) doit s'annuler en 1 par construction."

### Le graphique en version « dynamique »

Pour cet exercice nous allons directement passer à la version dynamique du graphique qui regroupe :
- le champ de vecteurs ;
- la solution approchée qui vérifie $y(t_0) = y_0$ trouvée par `odeint` (quand $y_0 = 0$ il s'agit de la fonction constante $y=0$, et dans les autres cas c'est la fonction $s_{t_0}$ qui est approximée) ;
- les « branches » manquant dus à la non-unicité de la solution.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -10, 10, -10, 10

# discrétisation du temps (100 instants dans [tmin,tmax])
t = np.linspace(tmin, tmax, 100)

@interact(t_0=(tmin, tmax, .5), y_0=(ymin, ymax, .5))
def dyn_solution(t_0=0, y_0=0):
    # taille de la figure
    plt.figure(figsize=(.5 * (tmax - tmin), .5 * (ymax - ymin)))
    # on détermine la fenêtre d'affichage
    plt.axis([tmin, tmax, ymin, ymax])
    # on trace les axes (qui coïncident avec l'isocline infinie)
    plt.plot([tmin, tmax], [0, 0], "y")
    plt.plot([0, 0], [ymin, ymax], "y")

    # on dessine le champ normalisé
    champ_normalise(f, tmin, tmax, ymin, ymax, width=.002)

    # La solution approchée, trouvée avec odeint
    # la partie de la courbe pour t dans [t_0,tmax] (le futur)
    futur = np.linspace(t_0, tmax, 50)
    v = odeint(f, y_0, futur)
    plt.plot(futur, v, "b")
    # la partie de la courbe pour t dans [tmin,t_0] (le passé)
    passe = np.linspace(t_0, tmin, 50)
    v = odeint(f, y_0, passe)
    plt.plot(passe, v, "g--")

    # les solutions non trouvées par odeint
    if y_0 == 0:
        # (t/3)^3
        s = ss(t_0)
        plt.plot(t, s(t), "r")
    elif y_0 != 0:
        # la solution constante 0
        plt.plot([tmin, tmax], [0, 0], "r")

    # le point de départ (la condition initiale)
    plt.plot(t_0, y_0, 'or')
    # le titre
    plt.title(f"Des solutions avec y({t_0})={y_0}.")
    # et on affiche tout ça
    plt.show()

### Exercice 4 (à faire)

Dans l'exercice 4 de la feuille 2, nous avons étudié l'équation non autonome
$$
    (1+t+t^2)y'+(2t+1)y = (1+t+t^2)^2. 
$$
Ainsi la fonction $f$ à définir est $f(y,t) =-\frac{1+2t}{1+t+t^2}y+(1+t+t^2)$.

Définissez cette fonction `f(y,t)` qui doit passer les tests.

In [ ]:
# la fonction f

# test
assert f(0, 0) == 1, "f(0,0) = 1"
assert f(1, 1) == 2, "f(1,1) = 2"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

### La solution exacte avec $y(0)=c$

Nous avons vu en TD que la solution qui vérifie $y(0)=c$ est
$$
s_c : t\mapsto \frac{c+t+t^2+t^3+t^4/2+t^5/5}{1+t+t^2}
$$

Définissez une fonction `ss(c)` qui retourne la fonction $s_c$. Cette fonction doit passer les tests.

In [ ]:
# la fonction ss qui retourne une fonction.

# si s = ss(1), alors s(0)=1 car s_c(0) = c
assert ss(1)(0) == 1, "s_c(0) = c"
assert np.shape(ss(1)(np.ones(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

### La représentation graphique

Adapter les représentations graphiques (dynamique, avec paramètres `t_0` et `y_0`) des exercices 3 et 4 à cet exercice. Le graphique doit représenter, en plus de la solution exacte `ss(y_0)` qui vérifie $y(0)=y_0$, la solution trouvée par `odeint` qui vérifie la condition initiale $y(t_0)=y_0$. 

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -10, 10, -5, 5



**Question :** Expliquer comment à partir de la représentation graphique du champ de vecteurs _(et non à partir de l'équation de départ)_ on voit qu'il ne s'agit pas d'une équation autonome.

## exercice 8 (à faire)

L'équation autonome étudié dans cet exercice est
$$
    y'=\sqrt{1+y^2}. 
$$
Ainsi la fonction $f$ à définir est $f(y,t) =\sqrt{1+y^2}$.

In [ ]:
# la fonction f

# test
assert f(0, 0) == 1, "f(0,*) = 1"
assert f(1, 1) == np.sqrt(2), "f(1,*) = sqrt(2)"
assert np.shape(f(np.ones(3), np.zeros(3))) == (3, ), "la fonction doit accepter des vecteurs comme paramètres"

### La solution en (0,1)

Nous avons vu en TD que la solution qui vérifie la condition initiale $y(0)=1$ est
$$
    s : t\mapsto \sinh(t+\operatorname{arcsinh}(1))
$$

Soit $s_c$ la solution qui vérifie $s_c(0) = c$. Définissez la fonction `ss(c)` qui retourne la solution qui vaut `c` en `0`. Cette fonction doit vérifier les tests.

In [ ]:
# la fonction s qui vérifie s(0) = 1

# si s=ss(c), alors s(0)=c
s = ss(1)
assert s(0) == 1, "si s=ss(1), alors s(0) doit être 1"
assert (s(np.array([0, np.arcsinh(-1)])) == np.array([1, 0])).all(), "s([0,arcsinh(-1)]) = [1,0]"

### La représentation graphique

Adapter les représentations graphiques des exercices 2 et 3 à cet exercice.

In [ ]:
# choix de la fenêtre
tmin, tmax, ymin, ymax = -10, 10, -5, 5



**Question :** Expliquer comment à partir de la représentation graphique du champ de vecteurs _(et non à partir de l'équation de départ)_ on voit qu'il s'agit d'une équation autonome.